In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from datacube import helpers

In [3]:
help(helpers.write_geotiff)

Help on function write_geotiff in module datacube.helpers:

write_geotiff(filename, dataset, profile_override=None, time_index=None)
    Write an ODC style xarray.Dataset to a GeoTIFF file.
    
    :param filename: Output filename
    :param dataset: xarray dataset containing one or more bands to write to a file.
    :param profile_override: option dict, overrides rasterio file creation options.
    :param time_index: DEPRECATED



In [4]:
#load modules
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

import rasterio
from datacube_stats.statistics import GeoMedian
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
from sklearn.ensemble import ExtraTreesClassifier
import datacube_stats
import os
# Replace '156' and 'ck9738' with the path to your own home directory on the VDI
s2aws = Datacube(config='/home/547/ck9738/datacube-s2.conf')

In [5]:
#define nessisary gotiff handeling fuctions

import xarray, rasterio, numpy as np
def numpy_to_xarray(array, geobox, name=None):
    """Utility to convert ndarray to DataArray, using a datacube.model.GeoBox"""
    coords=[xarray.Coordinate(x, geobox.coords[x].values, attrs=dict(units=geobox.coords[x].units)) 
            for x in geobox.dims]
    return xarray.DataArray(array, coords=coords, attrs=dict(crs=geobox.crs), name=name)

def geopandas_to_xarray(table, geobox, name=None):
    """Rasterise (with reprojection)"""
    array = rasterio.features.rasterize(shapes=table.to_crs(geobox.crs._crs.ExportToProj4()).geometry,
                                        out_shape=(geobox.height, geobox.width),
                                        transform=geobox.affine)
    return numpy_to_xarray(array, geobox, name)

def rasterfile_to_xarray(file, geobox, name=None, nodata=True, num_bands=(1), X_band= 1):
    """Blit like
    """
    with rasterio.open(file) as src:

        #|assert src.indexes == num_bands # assume single band
        band = rasterio.band(src, X_band) # do not attempt to read entire extent into memory
        array = np.empty((geobox.height, geobox.width), dtype=band.dtype)
        rasterio.warp.reproject(source=band,
                                destination=array,
                                dst_crs=geobox.crs.crs_str,
                                dst_transform=geobox.affine,
                                dst_nodata=nodata)
    return numpy_to_xarray(array, geobox, name)

In [6]:
"""#define area
# open  shapfile and data loat from that

shapefile_loc='/g/data/u46/users/ck9738/Datasets/woody_files/test_area_outline.shp'

#import project area shapefiles
project_area = gpd.read_file(shapefile_loc)

#convert the shapefile to GDA94 lat-long coords so we can query dc_load using lat long
project_area['geometry'] = project_area['geometry'].to_crs(epsg=4283)

#find the bounding box that contains all the queried projects
#addig 0.025 degrees to the bounding box to be certain the polygons are captured
coords = project_area.total_bounds
xmin, ymax, xmax, ymin =coords"""



"#define area\n# open  shapfile and data loat from that\n\nshapefile_loc='/g/data/u46/users/ck9738/Datasets/woody_files/test_area_outline.shp'\n\n#import project area shapefiles\nproject_area = gpd.read_file(shapefile_loc)\n\n#convert the shapefile to GDA94 lat-long coords so we can query dc_load using lat long\nproject_area['geometry'] = project_area['geometry'].to_crs(epsg=4283)\n\n#find the bounding box that contains all the queried projects\n#addig 0.025 degrees to the bounding box to be certain the polygons are captured\ncoords = project_area.total_bounds\nxmin, ymax, xmax, ymin =coords"

In [7]:
""" 
square 60 -149  'lat': (-33.107410, -33.216838),
near parks      'lon': (148.264306, 148.470191),

canberra east 
        'lat': (-35.219946, -35.404461),
        'lon': (149.12468, 149.413767),

square 60_-155, Barwang
        'lat': (-34.382385, -34.577689),
        'lon': (148.425899, 148.734517),
        
Lake Wombah 49 -129
        'lat': (-28.85916, -29.11364),
        'lon': (144.73454, 145.023037),

"""

" \nsquare 60 -149  'lat': (-33.107410, -33.216838),\nnear parks      'lon': (148.264306, 148.470191),\n\ncanberra east \n        'lat': (-35.219946, -35.404461),\n        'lon': (149.12468, 149.413767),\n\nsquare 60_-155, Barwang\n        'lat': (-34.382385, -34.577689),\n        'lon': (148.425899, 148.734517),\n        \nLake Wombah 49 -129\n        'lat': (-28.85916, -29.11364),\n        'lon': (144.73454, 145.023037),\n\n"

In [8]:
#load sentinal data
sensors= ['s2a_ard_granule'] #pick the sentinal satelites you want

bands_of_int =['red',
        'blue', 'green', 'rededge1', 
        'rededge2', 'rededge3', 'nir1', 'nir2', 'swir1', 'swir2', 'pixel_quality'
        ] #pick the sentinal bands that you want, here i am just using visible light    

query = {
        'lat': (-33.107410, -33.216838),
        'lon': (148.264306, 148.470191),
        'output_crs': 'EPSG:3577',
        'resolution': (-10, 10),
        'time':('2016-01-01', '2016-07-30')
        }
      
# use s2b_ard_granule 	 for S2B
data_sent = s2aws.load(product='s2a_ard_granule', measurements=bands_of_int, group_by='solar_day', **query)
clear_pixels = np.logical_and(data_sent.pixel_quality != 2, data_sent.pixel_quality != 3)

data_sent = data_sent.where(clear_pixels)


In [9]:
data_sent

<xarray.Dataset>
Dimensions:        (time: 7, x: 2049, y: 1457)
Coordinates:
  * time           (time) datetime64[ns] 2016-01-14T00:19:02.528000 ...
  * y              (y) float64 -3.707e+06 -3.707e+06 -3.707e+06 -3.707e+06 ...
  * x              (x) float64 1.502e+06 1.502e+06 1.502e+06 1.502e+06 ...
Data variables:
    red            (time, y, x) float64 1.412e+03 1.299e+03 1.309e+03 ...
    blue           (time, y, x) float64 940.0 900.0 901.0 959.0 1.05e+03 ...
    green          (time, y, x) float64 1.134e+03 1.053e+03 1.134e+03 ...
    rededge1       (time, y, x) float64 1.713e+03 1.73e+03 1.73e+03 ...
    rededge2       (time, y, x) float64 2.185e+03 2.187e+03 2.187e+03 ...
    rededge3       (time, y, x) float64 2.491e+03 2.483e+03 2.483e+03 ...
    nir1           (time, y, x) float64 2.695e+03 2.47e+03 2.686e+03 ...
    nir2           (time, y, x) float64 2.82e+03 2.725e+03 2.725e+03 ...
    swir1          (time, y, x) float64 3.564e+03 3.416e+03 3.416e+03 ...
    swir2       

In [10]:
#generate sentinal geomedian?
#cast the data to float numbers so geomedian calculation will work
data_sent_flt = data_sent.astype(float,casting='safe')

#create geomedian for data
Sentinal_gm=GeoMedian().compute(data_sent_flt)


KeyboardInterrupt: 

In [ ]:
Sentinal_gm

In [ ]:
#plot RGB image
#Prepare geomedian imagery for plotting

rgb = Sentinal_gm.to_array(dim='color').sel(color=[
    'red','green', 'blue']).transpose('y', 'x', 'color')
fake_saturation = 2000.0  #you may need to adjust this to get a clear image. higher numbers will make it darker, 
                                                                            #lower will make it ligher
rgb = rgb.astype('double')
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

fig = plt.figure(figsize =(8,8)) #edit size of plot 
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05) #set border dimensions
fig.patch.set_facecolor('white') #make border white 
fig.patch.set_alpha(0.99)#ensure border white
plt.axis('off')#remove axis 
plt.title('geomedian of selected area') #add title
plt.imshow(scaled, interpolation = 'nearest') #create image

In [25]:
#transform into format to feed to algorithm
sent_array = xr.DataArray(Sentinal_gm, dims=('y','x'))
flat_gm = sent_array.stack(z=('y','x')) #we stack Y and X together to make the array 2D, where shape is 
                                                  # N_samples x N_features
    
#flat_gm = flat_gm.transpose()#transpose so sample numbers is first dimention. This is what the machine 

flat_gm#learning algarithm wants 

KeyError: 'x'

In [15]:
#load traning data
panda_training= pd.read_csv('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/shapefiles_for_train/ncas_truthed_landsat_surface_ref.csv',
                            usecols=(1,2,3,4,5,6,7,8,9,10,11))

panda_training

,class,red,blue,green,rededge1,rededge2,rededge3,nir1,nir2,swir1,swir2
0,0,789.072560,569.215527,733.625553,1136.248934,2014.036523,2252.125539,2699.660338,2557.503778,1963.906536,1281.260957
1,0,947.070197,634.705545,835.921456,1310.055294,2045.536003,2258.763755,2393.898680,2573.035571,2558.706461,1765.414404
2,0,796.652061,563.693762,744.871038,1309.820378,2048.701824,2261.317049,2473.074444,2576.432795,2560.947171,1766.591793
3,0,660.178052,498.744516,667.211841,1083.610565,1912.477739,2111.790312,2552.511028,2390.650990,2054.596270,1282.232502
4,0,647.081762,484.738824,651.137623,1074.626143,1905.010082,2103.464739,2269.341989,2384.009886,2050.405421,1277.239845
5,0,513.159457,390.138896,507.850624,912.778484,1628.503021,1850.538698,1683.504285,2071.987647,1700.471800,1030.952276
6,1,562.442501,412.649401,531.697669,911.407313,1624.491464,1849.799762,2036.254544,2077.361592,1715.121627,1040.897667
7,1,556.414989,407.414076,530.167829,821.161434,1553.572039,1777.588231,1993.366529,2010.171728,1440.359115,843.396217
8,1,494.399595,369.002308,468.365681,815.449215,1549.396754,1771.562660,1800.859597,2005.717843,1435.026338,839.249374
9,1,578.874830,438.185981,552.196327,1022.777000,1877.362858,2105.203088,2293.332906,2380.293582,1717.015924,1093.678952


In [16]:
training_features = panda_training.as_matrix(columns=['red','blue', 'green', 'rededge1','rededge2', 'rededge3', 'nir1', 'nir2', 'swir1', 'swir2']) #create Array of features for training
training_class = panda_training.as_matrix(columns=['class']) #create Array of classification answer for training
training_class

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [17]:
#define machine learning algarithm 
classify = ExtraTreesClassifier(bootstrap=False, max_depth=10, random_state=0)

classify.fit(training_features,training_class)#fit the moddle to the training data
#train

/g/data/v10/public/modules/dea-env/20180405/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=10, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
#classify area
results=classify.predict(flat_gm)

/g/data/v10/public/modules/dea-env/20180405/lib/python3.6/site-packages/sklearn/utils/validation.py:433: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  array = np.array(array, dtype=dtype, order=order, copy=copy)
/g/data/v10/public/modules/dea-env/20180405/lib/python3.6/site-packages/sklearn/utils/validation.py:433: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  array = np.array(array, dtype=dtype, order=order, copy=copy)


ValueError: could not convert string to float: 'red'

In [ ]:
#return results back to original shape
results_back = results.reshape(ori_y,ori_x)
results_xr = xr.DataArray(results_back,dims=('y', 'x'),name='results')
results_xr = results_xr.astype('int16')
results_xr

In [ ]:
fig, ax = plt.subplots(figsize = (12,12))
cax = ax.imshow(results_xr)

plt.show()

In [ ]:
#add classification results to dataset
##replace with Geomediam 
Sentinal_time["class"] = results_xr
Sentinal_time
#add smad/green to dataset?

In [ ]:
#save dataset as netCDF to look at in QGIS
#write_geotiff(filename, dataset, profile_override=None, time_index=None)
helpers.write_geotiff('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/lst7_train_lake_wombah.tif',Sentinal_time)